# Analyze results with histograms and plots made from scratch
October 27, 2020

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
from ipywidgets import *

In [ ]:
%matplotlib widget

In [ ]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/')
from modules_image_analysis import *

In [ ]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


## Read data

In [ ]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

In [ ]:
dict1={'128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/',
      '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/512sq/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)


In [ ]:
# parent_dir=u.result
parent_dir=dict1['128']
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

In [ ]:
result=w.result
result_dir=parent_dir+result
print(result_dir)


In [ ]:
# ### Read loss data
# with open (result_dir+'/metrics.pickle','rb') as f:
#     metrics_dict=pickle.load(f)

# metrics_dict

# df_metrics=pd.DataFrame([])
# for key in metrics_dict.keys(): df_metrics[key]=metrics_dict[key]
# df_metrics['step']=np.arange(df_metrics.shape[0])
# # df_metrics=df_metrics.dropna()

## Plot Losses

In [ ]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)

In [ ]:
df_metrics.step.values.shape,df_metrics.G_full.values.shape

In [ ]:
df_metrics.head(10)

In [ ]:
df_metrics[df_metrics.step%50==0].head(10)

In [ ]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

In [ ]:
chi=df_metrics.dropna().quantile(q=0.05,axis=0)['hist_chi']
print(chi)
df_metrics[df_metrics['hist_chi']<=chi].sort_values(by=['hist_chi']).head(10)

In [ ]:
display(df_metrics.sort_values(by=['hist_chi']).head(8))
display(df_metrics.sort_values(by=['spec_chi']).head(8))

## Read stored chi-squares for images

In [ ]:
df=pd.read_pickle(result_dir+'/df_processed.pkle')
df[['epoch','step']]=df[['epoch','step']].astype(int)

chi1=df.quantile(q=0.2,axis=0)['chi_1']
chi_spec=df.quantile(q=0.2,axis=0)['chi_spec1']

df[(df.chi_spec1<chi_spec)&(df.chi_1<chi1)].sort_values(by=['chi_spec1','chi_1']).head(10)
# df[df.chi_1<chi1].sort_values(by=['chi_1']).head(5)


In [ ]:
df[(df.chi_spec1<chi_spec)&(df.chi_1<chi1)].sort_values(by=['chi_1','chi_spec1']).head(5)
# df[df.chi_spec1<chi_spec].sort_values(by=['chi_spec1']).head(5)


In [ ]:
step_list=[4111,5771]

print([(j.epoch,j.step) for i,j in df[df.step.isin(step_list)][['epoch','step']].iterrows()])

In [ ]:
df_metrics[df_metrics.step.isin(step_list)]


In [ ]:
df[df.step.isin(step_list)][['epoch','step','chi_1','chi_spec1']]


In [ ]:
df[(df.step>1000) & (df.step<1040)][['epoch','step','chi_1','chi_spec1']]

In [ ]:
df_metrics.query('step>1000 & step<1040 & step%2==0')

## Plot histogram and spectrum

In [ ]:

def f_widget_compare(sample_names,sample_dict,bkgnd_arr,Fig_type='pixel',rescale=True,log_scale=True,bins=25,mode='avg',normalize=True,crop=False):
    '''
    Module to make widget plots for pixel intensity or spectrum comparison for multiple sample sets
    '''
    
    
    if crop: # Crop out large pixel values
        for key in sample_names:
            print(sample_dict[key].shape)
            sample_dict[key]=np.array([arr for arr in sample_dict[key] if np.max(arr)<=0.994])
            print(sample_dict[key].shape)
    
    img_list=[sample_dict[key] for key in sample_names]
    label_list=list(sample_names)
        
    hist_range=(0,0.996)
    
    if rescale: 
        for count,img in enumerate(img_list):
            img_list[count]=f_invtransform(img)
        hist_range=(0,2000)
        bkgnd_arr=f_invtransform(bkgnd_arr)
    
    assert Fig_type in ['pixel','spectrum'],"Invalid mode %s"%(mode)
    
    if Fig_type=='pixel':
        f_compare_pixel_intensity(img_lst=img_list,label_lst=label_list,bkgnd_arr=bkgnd_arr,normalize=normalize,log_scale=log_scale, mode=mode,bins=bins,hist_range=hist_range)
    elif Fig_type=='spectrum':
        f_compare_spectrum(img_lst=img_list,label_lst=label_list,bkgnd_arr=bkgnd_arr,log_scale=log_scale)



## Plot spectrum

In [ ]:
dict_samples={}

In [ ]:
# keras image for comparison
f1='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/exagan1/run5_fixed_cosmology/models/gen_imgs.npy'
img_2=np.expand_dims(np.load(f1)[:500],axis=1)[:,0,:,:]
dict_samples['keras']=img_2

In [ ]:
# Read input data

fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
samples=np.load(fname,mmap_mode='r+')[:2000]
samples=samples[:,0,:,:]
print(samples.shape)
# dict_samples['val input']=samples

In [ ]:
for step in step_list:
    fname='*gen_img_epoch-*_step-{0}.npy'.format(step)
    fle=glob.glob(result_dir+'/images/'+fname)[0]
    imgs=np.load(fle)
#     img_list.append(imgs)
#     labels_list.append('Step:',step)
    strg='step:%s'%(step)
    dict_samples[strg]=imgs
    

In [ ]:
bkgnd=samples

# bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
# bins=f_transform(bins)   ### scale to (-1,1) 
bins=50
interact_manual(f_widget_compare,sample_dict=fixed(dict_samples),
                sample_names=SelectMultiple(options=dict_samples.keys()), bkgnd_arr=fixed(bkgnd),
                Fig_type=ToggleButtons(options=['pixel','spectrum']),bins=fixed(bins),mode=['avg','simple'])

## Plot images

In [ ]:
f_plot_grid(bkgnd[100:118],cols=6,fig_size=(6,3))
f_plot_grid(dict_samples['step:39400'][40:58],cols=6,fig_size=(6,3))